<a href="https://colab.research.google.com/github/bibekyess/dacon-hansol-deco-challenge/blob/main/Solar_train_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

#### If you are running on local environment, please create a virtual environment with python-3.10 and torch2.1.0

In [2]:
!python --version

Python 3.10.12


In [2]:
import torch
torch.__version__, torch.cuda.is_available()

('2.1.0+cu121', True)

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [15]:
!pip install -qU bitsandbytes wandb peft tqdm sentence-transformers pandas accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper

In [2]:
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import os
import wandb

from transformers import PreTrainedTokenizerFast, AdamW, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm


## Data Preprocessing

In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/DACON_wallpaper_contest_data

/content/drive/MyDrive/DACON_wallpaper_contest_data


In [4]:
import csv

csv_file_path = "/content/drive/MyDrive/DACON_wallpaper_contest_data/pre-processed-data/augmented_train.csv"
with open(csv_file_path, 'r', newline='') as csv_file:
    # Create a CSV reader object
    csv_reader = csv.reader(csv_file)

    keys = []
    wallpaper_qna_dataset = []
    # Iterate over each row in the CSV file
    for idx, row in enumerate(csv_reader):
        if idx==0:
            keys=row
        else:
            triplet_entry = {}
            for value_idx, value in enumerate(row):
                triplet_entry[keys[value_idx]] = value
            wallpaper_qna_dataset.append(triplet_entry)
    print(keys)

['id', 'category', 'Question', 'Answer', 'Context']


In [5]:
wallpaper_qna_dataset[0]

{'id': 'TRAIN_000',
 'category': '건축구조',
 'Question': '면진장치가 뭐야? 면진장치에 사용되는 주요 기술은 무엇인가요?',
 'Answer': '면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다.',
 'Context': '면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다. 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다. 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.'}

In [6]:
from datasets import Dataset, load_dataset

wallpaper_qna_dataset = Dataset.from_list(wallpaper_qna_dataset)

In [7]:
wallpaper_qna_dataset

Dataset({
    features: ['id', 'category', 'Question', 'Answer', 'Context'],
    num_rows: 644
})

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [9]:
# """Don't use Orion because of security issue"""
# model_id = "OrionStarAI/Orion-14B-Chat"
# model_id = "OrionStarAI/Orion-14B-Chat-Int4" #No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

model_id = "upstage/SOLAR-10.7B-Instruct-v1.0" # Chat Template is available

# model_id = "mistralai/Mistral-7B-Instruct-v0.2" # Chat Template is available

# This model is instruction tuned on multitask and is high on leaderboard
# model_id = "davidkim205/komt-solar-10.7b-sft-v3"

# This model is the highest and there is a system prompt also
# model_id = "Edentns/DataVortexS-10.7B-dpo-v1.11" # Follows Alpaca Chat format
# https://huggingface.co/Edentns/DataVortexS-10.7B-dpo-v1.11
# Resource that says that simple prompt is better than wordy prompt
# https://medium.com/towards-generative-ai/quantized-fine-tuning-llama-2-for-rag-question-answering-426114daa09a


In [10]:
# tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False, trust_remote_code=True, eos_token='</s>')
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, eos_token='</s>')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
SYSTEM_PROMPT_TEMPLATE = """\
벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.
"""
USER_PROMPT_TEMPLATE = """\
<보고서>
{CONTEXT}
</보고서>
지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
<질문>
{QUESTION}
</질문>
"""

def create_instruction(dataset_sample, return_response=True):
  system_prompt = SYSTEM_PROMPT_TEMPLATE
  user_prompt = USER_PROMPT_TEMPLATE.format(CONTEXT=dataset_sample['Context'], QUESTION=dataset_sample['Question'])
  chat = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": dataset_sample['Answer']}
  ]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False)
  return prompt


create_instruction(wallpaper_qna_dataset[0])


'### System:\n벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.\n\n\n### User:\n<보고서>\n면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다. 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다. 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.\n</보고서>\n지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.\n<질문>\n면진장치가 뭐야? 면진장치에 사용되는 주요 기술은 무엇인가요?\n</질문>\n\n\n### Assistant:\n면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다.'

In [12]:
prepared_wallpaper_qna_dataset = wallpaper_qna_dataset.train_test_split(test_size=0.1)
prepared_wallpaper_qna_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'Question', 'Answer', 'Context'],
        num_rows: 579
    })
    test: Dataset({
        features: ['id', 'category', 'Question', 'Answer', 'Context'],
        num_rows: 65
    })
})

### Designing Prompt Template (No need to run this):
Instresting finding is that we can directly find the prompt template underlining of each tokenizer.

In [29]:
# # It seems a good idea to add custom tokens like <보고서> and <질문> to make a model be specific to our task

# INSTRUCTION_PROMPT_TEMPLATE = """\
# 벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.
# <보고서>
# {CONTEXT}
# </보고서>
# 지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
# <질문>
# {QUESTION}
# </질문>
# """

# RESPONSE_TEMPLATE="""{ANSWER}"""

# def create_instruction(context, question, answer, return_response=True):
#   instruction_prompt = INSTRUCTION_PROMPT_TEMPLATE.format(CONTEXT=context, QUESTION=question)
#   chat = [
#     {"role": "user", "content": instruction_prompt},
#     {"role": "assistant", "content": answer}
#   ]
#   prompt = tokenizer.apply_chat_template(chat, tokenize=False)
#   return prompt



In [13]:
# # model_id = "mistralai/Mistral-7B-Instruct-v0.2"
# chat = [
#   {"role": "system", "content": "You are a chatbot"},
#   {"role": "user", "content": "Hello, how are you?"},
#   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
# ]

# tokenizer.apply_chat_template(chat, tokenize=False)
"""
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] I'd like to show off how chat templating works! [/INST]
"""

"\n<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] I'd like to show off how chat templating works! [/INST]\n"

In [14]:
# model_id = "upstage/SOLAR-10.7B-Instruct-v1.0"

# chat = [
#   {"role": "user", "content": "Hello, how are you?"},
#   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
#   {"role": "user", "content": "I'd like to show off how chat templating works!"},
# ]

# tokenizer.apply_chat_template(chat, tokenize=False)
"""
### User:
Hello, how are you?

### Assistant:
I'm doing great. How can I help you today?### User:
I'd like to show off how chat templating works!

"""

"\n### User:\nHello, how are you?\n\n### Assistant:\nI'm doing great. How can I help you today?### User:\nI'd like to show off how chat templating works!\n\n"

In [15]:
# model_id = "OrionStarAI/Orion-14B-Chat-Int4"

# chat = [
#   {"role": "user", "content": "Hello, how are you?"},
#   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
#   {"role": "user", "content": "I'd like to show off how chat templating works!"},
# ]

# tokenizer.apply_chat_template(chat, tokenize=False)
"""
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing great. How can I help you today?<|im_end|>
<|im_start|>user
I'd like to show off how chat templating works!<|im_end|>
"""

"\n<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing great. How can I help you today?<|im_end|>\n<|im_start|>user\nI'd like to show off how chat templating works!<|im_end|>\n"

### Load Model and Fine

In [13]:
model_id

'upstage/SOLAR-10.7B-Instruct-v1.0'

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    

In [19]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


peft_config = LoraConfig(
    # r=64, # FIXME
    # lora_alpha=128, # practically it is suggested that lora_alpha is set to twice of r
    r=64, # FIXME put 64 if we have high resource
    lora_alpha=128, # practically it is suggested that lora_alpha is set to twice of r
    # In the original LoRA paper, they only add adapters on query and value (attention weights), but QLoRA to target all layers of the attention modules plus mlp too
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [20]:
# pwd

In [25]:
# mv /content/upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs/* /content/drive/MyDrive/DACON_wallpaper_contest_data/upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs/

mv: inter-device move failed: '/content/upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs/runs' to '/content/drive/MyDrive/DACON_wallpaper_contest_data/upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs/runs'; unable to remove target: Directory not empty


In [16]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    # save_strategy="epoch",
    save_strategy = "steps", # Change to `epoch` if we have enough resource
    save_steps = 50,
)

In [40]:
pip install -qU trl accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.5 MB/s eta 0:00:00


In [17]:
from trl import SFTTrainer

# max_seq_length=2048
max_seq_length=1024

trainer = SFTTrainer(
    model=base_model,
    args=args,
    train_dataset=prepared_wallpaper_qna_dataset["train"],
    formatting_func=create_instruction,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens" : False,
        "append_concat_token" : False,
    }
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:434: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
trainer.train()

wandb: Currently logged in as: bibekyess. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: 

In [19]:
trainer.save_model()

In [20]:
# args.output_dir

'upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs'

In [26]:
# del base_model
# del trainer
# torch.cuda.empty_cache()

In [16]:
""Interesting not working"""

# from peft import PeftModel


# model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/DACON_wallpaper_contest_data/upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs/checkpoint-100")

# messages = [{"role": "user", "content": "면진장치가 뭐야?"}]
# response = model.chat(tokenizer, messages, streaming=False)
# response

In [1]:
# from peft import AutoPeftModelForCausalLM
# import torch

# model = AutoPeftModelForCausalLM.from_pretrained(
#     "/content/drive/MyDrive/DACON_wallpaper_contest_data/upstage/SOLAR-10.7B-Instruct-v1.0-colab-outputs",
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     offload_folder = "offload/",
#     device_map="auto" #"cuda" if low_cpu_mem_usage=False
# )

# merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 47.06 MiB is free. Process 1154562 has 14.70 GiB memory in use. Of the allocated memory 14.46 GiB is allocated by PyTorch, and 141.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
# from transformers import pipeline

# text_pipe = pipeline("text-generation", model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

AttributeError: 'LlamaForCausalLM' object has no attribute 'chat'